## Import of libraries & dataset

### Import Libraries

In [ ]:
!pip install pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, isnull, col, sum, when, input_file_name, udf, to_timestamp, datediff
from pyspark.ml.feature import Bucketizer
import pandas as pd
import numpy as np
from pandas import Series
from numpy import var, mean, sqrt
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
# Define a schema for the CSV files
schema = StructType([
    StructField("ddl_case_id object", StringType(), True),
    StructField("year", StringType(), True),
    StructField("state_code", StringType(), True),
    StructField("dist_code", StringType(), True),
    StructField("court_no", StringType(), True),
    StructField("cino", StringType(), True),
    StructField("judge_position", StringType(), True),
    StructField("female_defendant", StringType(), True),
    StructField("female_petitioner", StringType(), True),
    StructField("female_adv_def", StringType(), True),
    StructField("female_adv_pet", StringType(), True),
    StructField("type_name", IntegerType(), True),
    StructField("purpose_name", StringType(), True),
    StructField("disp_name", StringType(), True),
    StructField("date_of_filing", StringType(), True),
    StructField("date_of_decision", StringType(), True),
    StructField("date_first_list", StringType(), True),
    StructField("date_last_list", StringType(), True),
    StructField("date_next_list", StringType(), True),
    StructField("act", StringType(), True),
    StructField("section", StringType(), True),
    StructField("billable_ipc", StringType(), True),
    StructField("number_sections_ipc", StringType(), True),
    StructField("criminal", StringType(), True),
    StructField("ddl_filing_judge_id", StringType(), True),
    StructField("ddl_decision_judge_id", StringType(), True),
    StructField("female_judge_filing", StringType(), True),
    StructField("start_date_filing", StringType(), True),
    StructField("end_date_filing", StringType(), True),
    StructField("female_judge_decision", StringType(), True),
    StructField("start_date_decision", StringType(), True),
    StructField("end_date_decision", StringType(), True),
    StructField("judgediff", StringType(), True),
    StructField("pendency_decision_filing", StringType(), True),
    StructField("pendency_first_filing", StringType(), True),
    StructField("pendency_decision_first", StringType(), True),
    StructField("pendency_next_last", StringType(), True),
    StructField("pendency_decision_last", StringType(), True),
    StructField("pendency_decision_next", StringType(), True),
    StructField("pendency_decision_filing_categories", StringType(), True),
    StructField("pendency_first_filing_categories", StringType(), True),
    StructField("filing_judge_tenure",StringType(), True),
    StructField("decision_judge_tenure", StringType(), True),
])

### Import of Cases 2010

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MAX_MEMORY = '45G'
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)

# Create a Spark session
spark = SparkSession.builder.appName("CSV Addition").config(conf=conf).getOrCreate()

In [ ]:

# Define a list of file paths for the CSV files
file_paths = ['/content/drive/MyDrive/Law/cases/2010/cases_2010_final.csv',
              '/content/drive/MyDrive/Law/cases/2011/cases_2011_final.csv',
              '/content/drive/MyDrive/Law/cases/2012/cases_2012_final.csv',
              '/content/drive/MyDrive/Law/cases/2013/cases_2013_final.csv',
              '/content/drive/MyDrive/Law/cases/2014/cases_2014_final.csv',
              '/content/drive/MyDrive/Law/cases/2015/cases_2015_final.csv',
              '/content/drive/MyDrive/Law/cases/2016/cases_2016_final.csv',
              '/content/drive/MyDrive/Law/cases/2017/cases_2017_final.csv',
              '/content/drive/MyDrive/Law/cases/2018/cases_2018_final.csv']

# Read the CSV files into a single dataframe
df_cases_orig = spark.read.csv(
    file_paths,
    schema=schema,
    header=True, # set to False if there's no header
    timestampFormat="yyyy-MM-dd'T'HH:mm:ss", # specify the format for timestamp columns
    #mode="FAILFAST" # fail fast if any errors occur while reading the CSV files
    mode="PERMISIVE" # fail fast if any errors occur while reading the CSV files
)

# Display the combined dataframe
df_cases_orig.show(5)

+--------------------+----+----------+---------+--------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+---------+------------+-------------------+--------+-------------------+---------------------+-------------------+-----------------+---------------+---------------------+-------------------+-----------------+---------+------------------------+---------------------+-----------------------+------------------+----------------------+----------------------+-----------------------------------+--------------------------------+-------------------+---------------------+
|  ddl_case_id object|year|state_code|dist_code|court_no|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_name|purpose_name|disp_name|      date_of_filing|    date_of_de

In [ ]:
# Define a list of file paths for the CSV files
file_paths = ['/content/drive/MyDrive/Law/religion/cases_religion_2018.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2017.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2016.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2015.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2014.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2013.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2012.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2011.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2010.csv']

# Read the CSV files into a single dataframe
df_cases_orig_religion = spark.read.csv(
    file_paths,
    #schema=schema,
    header=True, # set to False if there's no header
    #timestampFormat="yyyy-MM-dd'T'HH:mm:ss", # specify the format for timestamp columns
    #mode="FAILFAST" # fail fast if any errors occur while reading the CSV files
    mode="PERMISIVE" # fail fast if any errors occur while reading the CSV files
)

# Display the combined dataframe
df_cases_orig_religion.show(5)

+--------------------+----------------+-----------------+--------------+--------------+
|         ddl_case_id|muslim_defendant|muslim_petitioner|muslim_adv_def|muslim_adv_pet|
+--------------------+----------------+-----------------+--------------+--------------+
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|   -9998 unclear|      0 nonmuslim|         -9999|             0|
+--------------------+----------------+-----------------+--------------+--------------+
only showing top 5 rows



In [ ]:
df_cases_orig =  df_cases_orig.withColumn("act", col("act").cast("integer"))
df_cases_orig =  df_cases_orig.withColumn("section", col("section").cast("integer"))
#column_name ="section"
#null_count = df_cases_orig.select(sum(col(column_name).isNull().cast("int"))).collect()[0][0]
#print(null_count)

In [ ]:
total_rows = df_cases_orig.count()
print(total_rows)

79313266


In [ ]:
total_rows_religion = df_cases_orig_religion.count()
print(total_rows_religion)

80935944


In [ ]:
df_cases_orig.dtypes

[('ddl_case_id object', 'string'),
 ('year', 'string'),
 ('state_code', 'string'),
 ('dist_code', 'string'),
 ('court_no', 'string'),
 ('cino', 'string'),
 ('judge_position', 'string'),
 ('female_defendant', 'string'),
 ('female_petitioner', 'string'),
 ('female_adv_def', 'string'),
 ('female_adv_pet', 'string'),
 ('type_name', 'int'),
 ('purpose_name', 'string'),
 ('disp_name', 'string'),
 ('date_of_filing', 'string'),
 ('date_of_decision', 'string'),
 ('date_first_list', 'string'),
 ('date_last_list', 'string'),
 ('date_next_list', 'string'),
 ('act', 'int'),
 ('section', 'int'),
 ('billable_ipc', 'string'),
 ('number_sections_ipc', 'string'),
 ('criminal', 'string'),
 ('ddl_filing_judge_id', 'string'),
 ('ddl_decision_judge_id', 'string'),
 ('female_judge_filing', 'string'),
 ('start_date_filing', 'string'),
 ('end_date_filing', 'string'),
 ('female_judge_decision', 'string'),
 ('start_date_decision', 'string'),
 ('end_date_decision', 'string'),
 ('judgediff', 'string'),
 ('pendency

In [ ]:
old_header_name = "ddl_case_id object"
new_header_name = "ddl_case_id"

df_cases_orig = df_cases_orig.withColumnRenamed(old_header_name, new_header_name)

In [ ]:
df_cases_orig  = df_cases_orig.join(df_cases_orig_religion, on="ddl_case_id", how="left")

In [ ]:
df_cases_orig.show()

+--------------------+----+----------+---------+--------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------+------------+-------------------+--------+-------------------+---------------------+-------------------+-----------------+---------------+---------------------+-------------------+-----------------+---------+------------------------+---------------------+-----------------------+------------------+----------------------+----------------------+-----------------------------------+--------------------------------+-------------------+---------------------+----------------+-----------------+--------------+--------------+
|         ddl_case_id|year|state_code|dist_code|court_no|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_na

## Preprocessing & Feature Engineering

In [ ]:
df_cases_orig.printSchema()

root
 |-- ddl_case_id: string (nullable = true)
 |-- year: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- dist_code: string (nullable = true)
 |-- court_no: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- judge_position: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- purpose_name: string (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: string (nullable = true)
 |-- date_of_decision: string (nullable = true)
 |-- date_first_list: string (nullable = true)
 |-- date_last_list: string (nullable = true)
 |-- date_next_list: string (nullable = true)
 |-- act: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: string (nullable

In [ ]:
from pyspark.sql import functions as F

# Assuming df_cases_orig is your Spark DataFrame
df_cases_orig = df_cases_orig.withColumn("court_details", F.concat_ws("_", "state_code", "dist_code", "court_no"))
df_cases_orig = df_cases_orig.withColumn("state_district", F.concat_ws("_", "state_code", "dist_code"))

Filter all cases where decision is available (not required for pendency but for duration prediction)

In [ ]:
#df_cases_orig  = df_cases_orig [df_cases_orig ['pendency_decision_filing_categories'] != 'Undecided']

In [ ]:
#print(df_cases_orig.count())
# filter all cases where the case is Undecided and disp_name is not 26
rows_to_remove_mask = (
    (df_cases_orig["pendency_decision_filing_categories"] == "Undecided") &
    ((df_cases_orig["disp_name"] != "26") & (df_cases_orig["disp_name"] != "27"))
)

# Create a copy of the original DataFrame and drop the rows to be removed
df_cases_orig = df_cases_orig[~rows_to_remove_mask]
# Create a copy of the original DataFrame and drop the rows to be removed
#print(df_cases_orig.count())

In [ ]:
# create columns for binary classification problem
mapping_multi = {'upto 1 year': 'ontime' ,'1 to 2 years': 'ontime', '2 to 3 years': 'delayed','3 to 4 years': 'delayed','4 to 5 years': 'delayed', '5 to 6 years': 'delayed','6 to 7 years': 'delayed', '7 to 8 years': 'delayed','8 to 9 years': 'delayed', '9 to 10 years': 'delayed',  '10 to 11 years': 'delayed', 'Undecided': 'delayed'}
def map_decision(decision):
    return mapping_multi.get(decision, None)

# Register the UDF
map_decision_udf = F.udf(map_decision)

# Use the UDF to create a new column "pendency_model_multiyear"
df_cases_orig = df_cases_orig.withColumn("pendency_model_multiyear",
                                         map_decision_udf("pendency_decision_filing_categories"))

In [ ]:
# Assuming you have a Spark DataFrame called 'df_cases2010'
df_cases_orig = df_cases_orig.withColumn("start_date_filing", to_timestamp("start_date_filing", 'yyyy-MM-dd'))
df_cases_orig = df_cases_orig.withColumn("date_of_filing", to_timestamp("date_of_filing", "yyyy-MM-dd HH:mm:ssXXX"))

df_cases_orig = df_cases_orig.withColumn("filing_judge_tenure_at_filing", when(col("start_date_filing").isNull() | col("date_of_filing").isNull(), None)
                                        .otherwise(
                                            (datediff(col("date_of_filing"), col("start_date_filing"))/30).cast(IntegerType())
                                        ))

In [ ]:
unique_jt_values = df_cases_orig.select("filing_judge_tenure_at_filing").distinct()
#totcount = unique_jt_values.count()
#unique_jt_values.show(totcount, truncate=False)

In [ ]:
# Assuming you have a Spark DataFrame called 'df_cases2010'
df_cases_orig = df_cases_orig.withColumn("start_date_decision", to_timestamp("start_date_decision", 'yyyy-MM-dd'))
df_cases_orig = df_cases_orig.withColumn("date_of_decision", to_timestamp("date_of_decision", "yyyy-MM-dd HH:mm:ssXXX"))

df_cases_orig = df_cases_orig.withColumn("decision_judge_tenure_at_decision", when(col("start_date_decision").isNull() | col("date_of_decision").isNull(), None)
                                        .otherwise(
                                            (datediff(col("date_of_decision"), col("start_date_decision"))/30).cast(IntegerType())
                                        ))

In [ ]:
unique_jt_values = df_cases_orig.select("decision_judge_tenure_at_decision").distinct()
totcount = unique_jt_values.count()
unique_jt_values.show(totcount, truncate=False)

+---------------------------------+
|decision_judge_tenure_at_decision|
+---------------------------------+
|31                               |
|85                               |
|65                               |
|53                               |
|133                              |
|78                               |
|108                              |
|34                               |
|115                              |
|101                              |
|126                              |
|81                               |
|28                               |
|76                               |
|26                               |
|27                               |
|44                               |
|103                              |
|12                               |
|223                              |
|91                               |
|222                              |
|22                               |
|128                              |
|122                        

In [ ]:
print(df_cases_orig.count())

79293276


We will drop following features

* ddl_filing_judge_id
* ddl_decision_judge_id
* start_date_filing
* end_date_filing
* start_date_decision
* end_date_decision

The following are dropped as these dates are converted into categorical variables as periods

* disp_name

The following are dropped as they are just identifiers

* ddl_case_id object
* purpose_name

These variables are not included as deemed not important*

* pendency_decision_first
* pendency_decision_last
* pendency_decision_next
* pendency_next_last

Drop these as these are feature engineered for target variables

* pendency_decision_filing_categories

Drop these as these are feature engineered and used

* dist_code
* court_no


In [ ]:
#columns_to_drop = ['ddl_filing_judge_id','ddl_decision_judge_id','start_date_filing','end_date_filing','start_date_decision','end_date_decision',\
#                   'ddl_case_id','purpose_name','date_first_list','date_last_list','date_next_list',\
#                   'pendency_decision_first','pendency_decision_last', 'pendency_decision_next','pendency_next_last',\
#                   'dist_code','court_no']

columns_to_drop = ['ddl_filing_judge_id','ddl_decision_judge_id','start_date_filing','end_date_filing','start_date_decision','end_date_decision',\
                   'purpose_name','date_first_list','date_last_list','date_next_list',\
                   'pendency_decision_first','pendency_decision_last', 'pendency_decision_next','pendency_next_last',\
                   'dist_code','court_no']

# Create a copy of the dataframe and start preparing the same for prediction modelling
df_cases_modified_merge = df_cases_orig.drop(*columns_to_drop)

In [ ]:
df_cases_modified_merge.printSchema()

root
 |-- ddl_case_id: string (nullable = true)
 |-- year: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- judge_position: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- act: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pen

## Merge data from modified Act, Section and Judge Position File

In [ ]:
df_act = spark.read.csv("/content/drive/MyDrive/Law/keys/act_key.csv", header=True, inferSchema=True)
df_section = spark.read.csv("/content/drive/MyDrive/Law/keys/section_key.csv", header=True, inferSchema=True)
df_type = spark.read.csv("/content/drive/MyDrive/Law/keys/type_name_key.csv", header=True, inferSchema=True)

In [ ]:
df_act  = df_act.withColumnRenamed("count", "countact")
df_act = df_act.dropna(how='all')
df_act.show(5)


+--------------------+--------+----+
|               act_s|countact| act|
+--------------------+--------+----+
|                NULL| 11282.0|NULL|
|                   '| 11160.0|   1|
|                  ''| 20887.0|   2|
|'Tamil Nadu City ...|   371.0|   3|
|(BOMBAY) NATIONAL...|     2.0|   4|
+--------------------+--------+----+
only showing top 5 rows



In [ ]:
df_act.printSchema()

root
 |-- act_s: string (nullable = true)
 |-- countact: double (nullable = true)
 |-- act: integer (nullable = true)



In [ ]:
df_section=  df_section.withColumnRenamed("count", "countsection")
df_section = df_section.dropna(how='all')
df_section.show(5)

+--------------------+------------+-------+
|           section_s|countsection|section|
+--------------------+------------+-------+
|                NULL|     4834909|   NULL|
|!#11bofHindumarri...|           1|      1|
|              !3 (B)|           1|      2|
|          !3(1)(i-a)|           1|      3|
|               !3(3)|           1|      4|
+--------------------+------------+-------+
only showing top 5 rows



In [ ]:
df_section.printSchema()

root
 |-- section_s: string (nullable = true)
 |-- countsection: integer (nullable = true)
 |-- section: integer (nullable = true)



In [ ]:
df_type = df_type.drop("count")
df_type.show(5)

+----+---------+------------------+
|year|type_name|       type_name_s|
+----+---------+------------------+
|2010|        1|          (m) t.s.|
|2010|        2|              (sc)|
|2010|        3|        ..mact 166|
|2010|        4|04 complaint cases|
|2010|        5|  04 criminal case|
+----+---------+------------------+
only showing top 5 rows



In [ ]:
df_cases_modified_mergedact  = df_cases_modified_merge.join(df_act, on="act", how="left")

In [ ]:
df_cases_modified_mergedact.show(5)

+-----+--------------------+----+----------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+---------+-------------------+-------------------+-------+------------+-------------------+--------+-------------------+---------------------+---------+------------------------+---------------------+-----------------------------------+--------------------------------+-------------------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+---------+
|  act|         ddl_case_id|year|state_code|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_name|disp_name|     date_of_filing|   date_of_decision|section|billable_ipc|number_sections_ipc|criminal|female_judge_filing|female_judge_decision|judgediff|pendency_

In [ ]:
print(df_cases_modified_mergedact.count())

79293276


In [ ]:
df_cases_modified_mergedactsection  = df_cases_modified_mergedact.join(df_section,on="section", how="left")

In [ ]:
df_cases_modified_mergedactsection.show(5)

+-------+-----+--------------------+----+----------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+---------+-------------------+-------------------+------------+-------------------+--------+-------------------+---------------------+---------+------------------------+---------------------+-----------------------------------+--------------------------------+-------------------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+--------+--------------------+------------+
|section|  act|         ddl_case_id|year|state_code|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_name|disp_name|     date_of_filing|   date_of_decision|billable_ipc|number_sections_ipc|criminal|female_judge_filing|female_j

In [ ]:
df_cases_modified_mergedactsectiontype = df_cases_modified_mergedactsection.join(df_type, on = ["year","type_name"],  how="left")

In [ ]:
print(df_cases_modified_mergedactsection.count())

79293276


In [ ]:
df_judge = spark.read.csv("/content/drive/MyDrive/Law/judges_clean/judge_position_key.csv", header=True, inferSchema=True)

In [ ]:
def merge_judge_position(df):
    df  = df.join(df_judge,on="judge_position", how="left")
    df  = df.drop("judge_position")

    return df

In [ ]:
df_cases_modified_mergedactsectionjp = merge_judge_position(df_cases_modified_mergedactsectiontype)

In [ ]:
df_cases_modified_mergedactsectionjp.printSchema()

root
 |-- year: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- act: integer (nullable = true)
 |-- ddl_case_id: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: string (nullable = true)
 

In [ ]:
#Transform number_sections_ipc
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import DoubleType
bins = [-1, 1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 35, 40, float('inf')]
labelsipc = ['zero', 'one', 'two', 'three', 'four', '5 to 10', '10 to 15', '15 to 20',
          '20 to 25', '25 to 30', '30 to 35', '35 to 40', '40 plus']

# Define a UDF to map indices to labels
def map_index_to_label_ipc(index):
    if index is not None:
        return labelsipc[index]
    else:
        return "NA"

# Register the UDF
map_index_udf_ipc = udf(map_index_to_label_ipc, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("number_sections_ipc", col("number_sections_ipc").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="number_sections_ipc", outputCol="number_sections_ipc_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("number_sections_ipc_temp", col("number_sections_ipc_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("number_sections_ipc_category", map_index_udf_ipc(df_cases_modified_mergedactsectionjp["number_sections_ipc_temp"]))

In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("number_sections_ipc", "number_sections_ipc_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("number_sections_ipc")) | isnull(col("number_sections_ipc"))))
df_cases_sample.show()

+-------------------+----------------------------+
|number_sections_ipc|number_sections_ipc_category|
+-------------------+----------------------------+
|                3.0|                       three|
|                5.0|                     5 to 10|
|                2.0|                         two|
|                1.0|                         one|
|                1.0|                         one|
|                2.0|                         two|
|                4.0|                        four|
|                2.0|                         two|
|                1.0|                         one|
|                1.0|                         one|
|                1.0|                         one|
|                1.0|                         one|
|                1.0|                         one|
|                1.0|                         one|
|                1.0|                         one|
|                1.0|                         one|
|                1.0|          

In [ ]:
#Transform pendency
bins = [-1, 0, 1, 3, 6, 9, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, float('inf')]
labelspf = ['upto 1 month', '1 month', '1 to 3 months', '3 to 6 months', '6 month to 9 month' , '9 month to 1 year', '1 to 2 years', '2 to 3 years', '3 to 4 years', '4 to 5 years',
          '5 to 6 years', '6 to 7 years', '7 to 8 years', '8 to 9 years', '9 to 10 years',
          '10 year +']

# Define a UDF to map indices to labels
def map_index_to_label_pf(index):
    if index is not None:
        return labelspf[index]
    else:
        return "NA"

# Register the UDF
map_index_udf_pf = udf(map_index_to_label_pf, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("pendency_first_filing", col("pendency_first_filing").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="pendency_first_filing", outputCol="pendency_first_filing_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("pendency_first_filing_temp", col("pendency_first_filing_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("pendency_first_filing_category", map_index_udf_pf(df_cases_modified_mergedactsectionjp["pendency_first_filing_temp"]))


In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("pendency_first_filing", "pendency_first_filing_temp", "pendency_first_filing_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("pendency_first_filing")) | isnull(col("pendency_first_filing"))))
df_cases_sample.show()

+---------------------+--------------------------+------------------------------+
|pendency_first_filing|pendency_first_filing_temp|pendency_first_filing_category|
+---------------------+--------------------------+------------------------------+
|                  3.0|                         3|                 3 to 6 months|
|                  1.0|                         2|                 1 to 3 months|
|                 11.0|                         5|             9 month to 1 year|
|                  6.0|                         4|            6 month to 9 month|
|                  0.0|                         1|                       1 month|
|                 52.0|                         9|                  4 to 5 years|
|                  2.0|                         2|                 1 to 3 months|
|                  1.0|                         2|                 1 to 3 months|
|                  0.0|                         1|                       1 month|
|               

In [ ]:
#Transform filling_judge_tenure_at_filing
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit, expr
bins = [-1, 1, 3, 6, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 180, float('inf')]
labels = ['upto 1 months', '1 to 3 months', '3 to 6 months', '6 month to 1 year', '1 to 2 years', '2 to 3 years', '3 to 4 years', '4 to 5 years',
          '5 to 6 years', '6 to 7 years', '7 to 8 years', '8 to 9 years', '9 to 10 years',
          '10 to 15 years', '15 years plus']

# Define a UDF to map indices to labels
def map_index_to_label(index):
    if index is not None:
        return labels[index]
    else:
        return "NA"

# Register the UDF
map_index_udf = udf(map_index_to_label, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("filing_judge_tenure_at_filing", col("filing_judge_tenure_at_filing").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="filing_judge_tenure_at_filing", outputCol="filing_judge_tenure_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("filing_judge_tenure_temp", col("filing_judge_tenure_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("filing_judge_tenure_at_filing_category", map_index_udf(df_cases_modified_mergedactsectionjp["filing_judge_tenure_temp"]))

In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("filing_judge_tenure_at_filing", "filing_judge_tenure_at_filing_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("filing_judge_tenure_at_filing")) | isnull(col("filing_judge_tenure_at_filing_category"))))
df_cases_sample.show()

+-----------------------------+--------------------------------------+
|filing_judge_tenure_at_filing|filing_judge_tenure_at_filing_category|
+-----------------------------+--------------------------------------+
|                         NULL|                                    NA|
|                         NULL|                                    NA|
|                         NULL|                                    NA|
|                         NULL|                                    NA|
|                         NULL|                                    NA|
|                         NULL|                                    NA|
|                         NULL|                                    NA|
|                         24.0|                          2 to 3 years|
|                          8.0|                     6 month to 1 year|
|                         19.0|                          1 to 2 years|
|                          3.0|                         3 to 6 months|
|     

In [ ]:
#Transform decision_judge_tenure_at_decision
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit, expr
bins = [-1, 1, 3, 6, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 180, float('inf')]
labels = ['upto 1 months', '1 to 3 months', '3 to 6 months', '6 month to 1 year', '1 to 2 years', '2 to 3 years', '3 to 4 years', '4 to 5 years',
          '5 to 6 years', '6 to 7 years', '7 to 8 years', '8 to 9 years', '9 to 10 years',
          '10 to 15 years', '15 years plus']

# Define a UDF to map indices to labels
def map_index_to_label(index):
    if index is not None:
        return labels[index]
    else:
        return "NA"

# Register the UDF
map_index_udf = udf(map_index_to_label, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("decision_judge_tenure_at_decision", col("decision_judge_tenure_at_decision").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="decision_judge_tenure_at_decision", outputCol="decision_judge_tenure_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("decision_judge_tenure_temp", col("decision_judge_tenure_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("decision_judge_tenure_at_decision_category", map_index_udf(df_cases_modified_mergedactsectionjp["decision_judge_tenure_temp"]))

In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("decision_judge_tenure_at_decision", "decision_judge_tenure_at_decision_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("decision_judge_tenure_at_decision")) | isnull(col("decision_judge_tenure_at_decision_category"))))
df_cases_sample.show()

+---------------------------------+------------------------------------------+
|decision_judge_tenure_at_decision|decision_judge_tenure_at_decision_category|
+---------------------------------+------------------------------------------+
|                             NULL|                                        NA|
|                             NULL|                                        NA|
|                             NULL|                                        NA|
|                             NULL|                                        NA|
|                             NULL|                                        NA|
|                             NULL|                                        NA|
|                             NULL|                                        NA|
|                             10.0|                         6 month to 1 year|
|                             NULL|                                        NA|
|                             NULL|                 

In [ ]:
df_cases_modified_mergedactsectionjp.printSchema()

root
 |-- year: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- act: integer (nullable = true)
 |-- ddl_case_id: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: double (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 

In [ ]:
columns_to_drop_postmerge = ['pendency_first_filing_category', 'disp_name','type_name','number_sections_ipc',
                             'filing_judge_tenure','filing_judge_tenure_temp','pendency_first_filing_temp','number_sections_ipc_temp']

#columns_to_drop_postmerge = ['billable_ipc','type_name',','type_name','act','section','cino','disp_name','number_sections_ipc','pendency_decision_filling','pendency_first_filing','filing_judge_tenure','pendency_first_filing_categories','filing_judge_tenure_temp','pendency_first_filing_temp','number_sections_ipc_temp']
df_cases_modified_final = df_cases_modified_mergedactsectionjp.drop(*columns_to_drop_postmerge)

In [ ]:
df_cases_modified_final.printSchema()

root
 |-- year: string (nullable = true)
 |-- section: integer (nullable = true)
 |-- act: integer (nullable = true)
 |-- ddl_case_id: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 |-- pendency_decision_filing_categories: string (nullable = true)
 |-- pendency_first_filing_categories: string (nullable = true)
 |--

In [ ]:
#columns_to_drop_postmerge = ['section','act','year','cino', 'date_of_filing', 'judge_position', 'date_of_decision','pendency_first_filing_categories', 'decision_judge_tenure']
columns_to_drop_postmerge = ['section','act','year','cino', 'judge_position', 'pendency_first_filing_categories', 'decision_judge_tenure']
df_cases_modified_final = df_cases_modified_final.drop(*columns_to_drop_postmerge)

In [ ]:
df_cases_modified_final.printSchema()

root
 |-- ddl_case_id: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 |-- pendency_decision_filing_categories: string (nullable = true)
 |-- muslim_defendant: string (nullable = true)
 |-- muslim_petitioner: string (nullable = true)
 |-- muslim_adv_def: string (nullable = true)
 |-- muslim_adv_pet: string (nullable = true)
 |-- court_details: string 

In [ ]:
columns_to_drop_postmerge = ['pendency_decision_filing_categories','decision_judge_tenure','countact', 'count','decision_judge_tenure_temp', 'decision_judge_tenure_at_decision_category','filing_judge_tenure_at_filing_category']

df_cases_modified = df_cases_modified_final.drop(*columns_to_drop_postmerge)

In [ ]:
df_cases_modified.printSchema()

root
 |-- ddl_case_id: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 |-- muslim_defendant: string (nullable = true)
 |-- muslim_petitioner: string (nullable = true)
 |-- muslim_adv_def: string (nullable = true)
 |-- muslim_adv_pet: string (nullable = true)
 |-- court_details: string (nullable = false)
 |-- state_district: string (nullable = false)
 

In [ ]:
df_cases_modified= df_cases_modified.fillna("NA")

In [ ]:
df_cases_modified = df_cases_modified.filter(col("criminal") == 1)


In [ ]:
print(df_cases_modified.count())


24205125


In [ ]:
df_cases_modified.show(20)

+--------------------+----------+----------------+-----------------+--------------+--------------+-------------------+-------------------+------------+--------+-------------------+---------------------+---------+------------------------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+---------+------------+-----------+--------------------+----------------------------+
|         ddl_case_id|state_code|female_defendant|female_petitioner|female_adv_def|female_adv_pet|     date_of_filing|   date_of_decision|billable_ipc|criminal|female_judge_filing|female_judge_decision|judgediff|pendency_decision_filing|pendency_first_filing|muslim_defendant|muslim_petitioner|muslim_adv_def|muslim_adv_pet|court_details|state_district|pendency_model_multiyear|filing_judge_tenure_at_filing|decision_judge_tenure_at_decision|         

In [ ]:
df_cases_modified.printSchema()

root
 |-- ddl_case_id: string (nullable = false)
 |-- state_code: string (nullable = false)
 |-- female_defendant: string (nullable = false)
 |-- female_petitioner: string (nullable = false)
 |-- female_adv_def: string (nullable = false)
 |-- female_adv_pet: string (nullable = false)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = false)
 |-- criminal: string (nullable = false)
 |-- female_judge_filing: string (nullable = false)
 |-- female_judge_decision: string (nullable = false)
 |-- judgediff: string (nullable = false)
 |-- pendency_decision_filing: string (nullable = false)
 |-- pendency_first_filing: double (nullable = true)
 |-- muslim_defendant: string (nullable = false)
 |-- muslim_petitioner: string (nullable = false)
 |-- muslim_adv_def: string (nullable = false)
 |-- muslim_adv_pet: string (nullable = false)
 |-- court_details: string (nullable = false)
 |-- state_district: string (nul

In [ ]:
# Covid Impacted Cases Remove
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType
from pyspark.sql import functions as F

# Define cutoff dates as timestamp literals
cutoff_filing_date = F.lit("2018-03-25").cast(TimestampType())
cutoff_decision_date = F.lit("2020-03-25").cast(TimestampType())

# Apply correct parentheses to separate logical clauses
df_filtered = df_cases_modified.filter(
    ((col("date_of_filing") > cutoff_filing_date) & (col("date_of_decision").isNull())) |
    (col("date_of_decision") > cutoff_decision_date)
)

# Count filtered rows
filtered_count = df_filtered.count()
df_cases_modified = df_cases_modified.subtract(df_filtered)
df_cases_modified.count()

22389955

In [ ]:
#Filter out undecided cases delayed
df_cases_modified = df_cases_modified[
    (df_cases_modified['pendency_decision_filing'] > 24) &
    (df_cases_modified['pendency_decision_filing'] < 150)
]
df_cases_modified.count()

3035697

In [ ]:
columns_to_drop = ['countsection','criminal', 'date_of_decision','date_of_filing','ddl_case_id']
#columns_to_drop = ['countsection','pendency_decision_filing','criminal']

# Create a copy of the dataframe and start preparing the same for prediction modelling
df_cases_modified = df_cases_modified.drop(*columns_to_drop)

In [ ]:
df_cases_modified.printSchema()

root
 |-- state_code: string (nullable = false)
 |-- female_defendant: string (nullable = false)
 |-- female_petitioner: string (nullable = false)
 |-- female_adv_def: string (nullable = false)
 |-- female_adv_pet: string (nullable = false)
 |-- billable_ipc: string (nullable = false)
 |-- female_judge_filing: string (nullable = false)
 |-- female_judge_decision: string (nullable = false)
 |-- judgediff: string (nullable = false)
 |-- pendency_decision_filing: string (nullable = false)
 |-- pendency_first_filing: double (nullable = true)
 |-- muslim_defendant: string (nullable = false)
 |-- muslim_petitioner: string (nullable = false)
 |-- muslim_adv_def: string (nullable = false)
 |-- muslim_adv_pet: string (nullable = false)
 |-- court_details: string (nullable = false)
 |-- state_district: string (nullable = false)
 |-- pendency_model_multiyear: string (nullable = false)
 |-- filing_judge_tenure_at_filing: double (nullable = true)
 |-- decision_judge_tenure_at_decision: double (null

In [ ]:
state_code_df = df_cases_modified.groupBy("pendency_decision_filing").count()
totcount = state_code_df.count()
state_code_df.show(totcount, truncate=False)
print(totcount)

+------------------------+------+
|pendency_decision_filing|count |
+------------------------+------+
|51                      |44402 |
|54                      |38372 |
|101                     |3182  |
|29                      |107395|
|69                      |21692 |
|42                      |62798 |
|112                     |374   |
|73                      |17935 |
|87                      |9181  |
|64                      |26971 |
|30                      |103568|
|113                     |354   |
|34                      |86433 |
|59                      |31964 |
|28                      |112505|
|85                      |10303 |
|35                      |83236 |
|52                      |42195 |
|71                      |19929 |
|98                      |4375  |
|47                      |51244 |
|99                      |3886  |
|107                     |1214  |
|110                     |481   |
|96                      |5248  |
|43                      |60891 |
|31           

In [ ]:
state_code_df = df_cases_modified.groupBy("type_name_s").count()
totcount = state_code_df.count()
state_code_df.show(totcount, truncate=False)
print(totcount)

+-------------------------------------------------+-------+
|type_name_s                                      |count  |
+-------------------------------------------------+-------+
|spl.cri.m.a.                                     |477    |
|crl misc                                         |4      |
|mact misc                                        |4      |
|misc. ( criminal)                                |102    |
|g r case ( summon)                               |99     |
|summons private cases ss                         |93     |
|mv act                                           |70     |
|misc. civil appeal                               |36     |
|criminal misc. petition                          |12     |
|u.c. case                                        |24     |
|cri.reg.                                         |301    |
|cri case                                         |1089   |
|crl.t                                            |47     |
|b.misc case                            

In [ ]:
jp_df = df_cases_modified.groupBy("pendency_model_multiyear").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+------------------------+-------+
|pendency_model_multiyear|count  |
+------------------------+-------+
|delayed                 |3035697|
+------------------------+-------+

1


In [ ]:
jp_df = df_cases_modified.groupBy("pendency_first_filing").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------------------+------+
|pendency_first_filing|count |
+---------------------+------+
|8.0                  |45659 |
|67.0                 |3627  |
|70.0                 |3061  |
|0.0                  |510758|
|69.0                 |3184  |
|7.0                  |52182 |
|112.0                |61    |
|108.0                |127   |
|88.0                 |1139  |
|49.0                 |10464 |
|101.0                |303   |
|98.0                 |389   |
|116.0                |20    |
|29.0                 |27041 |
|107.0                |153   |
|64.0                 |4223  |
|75.0                 |2235  |
|47.0                 |11301 |
|42.0                 |14884 |
|44.0                 |13320 |
|35.0                 |20503 |
|62.0                 |4940  |
|96.0                 |563   |
|NULL                 |50800 |
|18.0                 |32233 |
|80.0                 |1667  |
|86.0                 |1170  |
|120.0                |2     |
|1.0                  |402737|
|39.0   

In [ ]:
jp_df = df_cases_modified.groupBy("filing_judge_tenure_at_filing").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-----------------------------+-------+
|filing_judge_tenure_at_filing|count  |
+-----------------------------+-------+
|147.0                        |32     |
|170.0                        |1      |
|8.0                          |24992  |
|70.0                         |451    |
|67.0                         |598    |
|0.0                          |51692  |
|69.0                         |434    |
|206.0                        |2      |
|7.0                          |28567  |
|142.0                        |32     |
|112.0                        |44     |
|154.0                        |16     |
|124.0                        |19     |
|235.0                        |1      |
|108.0                        |49     |
|88.0                         |154    |
|49.0                         |1341   |
|101.0                        |57     |
|98.0                         |98     |
|116.0                        |31     |
|29.0                         |6974   |
|107.0                        |91     |


In [ ]:
jp_df = df_cases_modified.groupBy("decision_judge_tenure_at_decision").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------------------------------+-------+
|decision_judge_tenure_at_decision|count  |
+---------------------------------+-------+
|8.0                              |55065  |
|67.0                             |560    |
|70.0                             |476    |
|0.0                              |74889  |
|69.0                             |558    |
|7.0                              |61748  |
|112.0                            |74     |
|154.0                            |1      |
|124.0                            |36     |
|128.0                            |9      |
|108.0                            |57     |
|88.0                             |74     |
|49.0                             |938    |
|101.0                            |66     |
|98.0                             |106    |
|238.0                            |6      |
|116.0                            |25     |
|29.0                             |12252  |
|107.0                            |67     |
|64.0                           

In [ ]:
state_code_df = df_cases_modified.groupBy("state_code").count()
totcount = state_code_df.count()
state_code_df.show(totcount, truncate=False)
print(totcount)

+----------+------+
|state_code|count |
+----------+------+
|7         |81755 |
|15        |10578 |
|11        |56637 |
|29        |139489|
|3         |157061|
|30        |2681  |
|8         |227354|
|22        |66953 |
|16        |89906 |
|5         |12998 |
|31        |391   |
|18        |57946 |
|27        |3863  |
|17        |234045|
|26        |41705 |
|6         |102577|
|19        |173   |
|23        |277372|
|25        |510   |
|9         |143548|
|24        |417   |
|32        |1084  |
|1         |514280|
|20        |8544  |
|10        |86126 |
|4         |200642|
|12        |1307  |
|13        |283815|
|14        |85571 |
|21        |854   |
|2         |145505|
|33        |10    |
+----------+------+

32


In [ ]:
jp_df = df_cases_modified.groupBy("state_district").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+--------------+-----+
|state_district|count|
+--------------+-----+
|23_15         |8121 |
|23_27         |6277 |
|8_6           |15994|
|10_32         |4287 |
|11_11         |2421 |
|22_13         |1226 |
|1_30          |21028|
|7_15          |2264 |
|13_58         |8251 |
|5_4           |1336 |
|6_12          |3913 |
|2_7           |5923 |
|2_23          |11578|
|10_16         |694  |
|17_42         |2954 |
|8_19          |3150 |
|11_21         |3013 |
|11_17         |1441 |
|26_8          |3846 |
|14_11         |2034 |
|29_7          |8253 |
|10_19         |637  |
|8_30          |5021 |
|1_21          |28694|
|10_12         |3044 |
|2_19          |4853 |
|1_33          |1427 |
|20_3          |1132 |
|5_11          |1626 |
|20_5          |438  |
|22_15         |1143 |
|10_25         |1634 |
|16_19         |2905 |
|23_44         |7354 |
|7_2           |3981 |
|10_27         |5246 |
|8_14          |4513 |
|23_16         |4262 |
|18_19         |446  |
|12_11         |8    |
|17_15     

In [ ]:
jp_df = df_cases_modified.groupBy("court_details").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-------------+-----+
|court_details|count|
+-------------+-----+
|16_7_3       |1543 |
|23_1_4       |509  |
|13_21_5      |439  |
|10_13_4      |30   |
|9_26_13      |121  |
|10_9_12      |491  |
|6_9_11       |312  |
|29_3_18      |27   |
|23_49_2      |110  |
|10_28_14     |6    |
|17_39_6      |548  |
|11_7_9       |20   |
|13_14_2      |482  |
|14_9_6       |1075 |
|17_13_12     |11   |
|1_9_15       |751  |
|9_22_33      |207  |
|17_24_7      |1051 |
|22_8_8       |53   |
|2_11_21      |833  |
|10_4_23      |230  |
|17_27_17     |21   |
|3_26_20      |495  |
|3_14_1       |292  |
|22_14_1      |296  |
|3_7_4        |21   |
|2_23_19      |1069 |
|1_5_6        |826  |
|6_13_4       |5086 |
|29_3_4       |887  |
|29_8_11      |1057 |
|1_8_9        |1144 |
|9_13_16      |362  |
|17_23_16     |491  |
|4_9_23       |2115 |
|18_18_3      |352  |
|1_34_4       |207  |
|10_24_17     |589  |
|2_17_5       |770  |
|1_30_3       |1    |
|3_23_14      |82   |
|23_39_1      |2727 |
|3_18_1   

In [ ]:
jp_df = df_cases_modified.groupBy("female_judge_filing").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-------------------+-------+
|female_judge_filing|count  |
+-------------------+-------+
|NA                 |2331379|
|1 female           |187925 |
|0 nonfemale        |495716 |
|-9998 unclear      |20677  |
+-------------------+-------+

4


In [ ]:
jp_df = df_cases_modified.groupBy("female_judge_decision").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------------------+-------+
|female_judge_decision|count  |
+---------------------+-------+
|NA                   |1733401|
|1 female             |394748 |
|0 nonfemale          |860755 |
|-9998 unclear        |46793  |
+---------------------+-------+

4


In [ ]:
jp_df = df_cases_modified.groupBy("judgediff").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------+-------+
|judgediff|count  |
+---------+-------+
|1.0      |569233 |
|NA       |2403595|
|0.0      |62869  |
+---------+-------+

3


In [ ]:
jp_df = df_cases_modified.groupBy("billable_ipc").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-------------------------+-------+
|billable_ipc             |count  |
+-------------------------+-------+
|NA                       |1194506|
|not applicable to section|58189  |
|unclear, need to check   |3647   |
|depends                  |61699  |
|bailable                 |1048828|
|non-bailable             |668828 |
+-------------------------+-------+

6


In [82]:
jp_df = df_cases_modified.groupBy("number_sections_ipc_category").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+----------------------------+-------+
|number_sections_ipc_category|count  |
+----------------------------+-------+
|10 to 15                    |9233   |
|two                         |400258 |
|5 to 10                     |279210 |
|NA                          |586658 |
|four                        |260841 |
|one                         |1125313|
|three                       |373628 |
|25 to 30                    |18     |
|20 to 25                    |108    |
|15 to 20                    |429    |
|40 plus                     |1      |
+----------------------------+-------+

11


In [83]:
jp_df = df_cases_modified.groupBy("judge_position_s").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+--------------------------------------------------+------+
|judge_position_s                                  |count |
+--------------------------------------------------+------+
|cantonment court                                  |1524  |
|lunawada                                          |886   |
|spcl court - bharatpur district hq                |362   |
|jjb dungarpur district hq                         |23    |
|jjb criminal                                      |33    |
|vaghaicourt                                       |84    |
|4-additional district judge                       |39    |
|additional district judge                         |20670 |
|additional court                                  |3339  |
|jcj court miryalaguda                             |1730  |
|jjb bharatpur district hq                         |42    |
|jjb court complex                                 |54    |
|g.n.roopwas taluka criminal                       |3     |
|pjcj cum xxv addl.mm at ibrahimpatnam  

In [ ]:
jp_df = df_cases_modified.groupBy("female_defendant").count()
jp_df.show()

In [85]:
jp_df = df_cases_modified.groupBy("female_petitioner").count()
jp_df.show()

+------------------+-------+
| female_petitioner|  count|
+------------------+-------+
|-9999 missing name|   5752|
|          1 female| 450149|
|     -9998 unclear|1850604|
|            0 male| 729192|
+------------------+-------+



In [ ]:
jp_df = df_cases_modified.groupBy("female_adv_def").count()
jp_df.show()

In [ ]:
jp_df = df_cases_modified.groupBy("female_adv_pet").count()
jp_df.show()

In [88]:
jp_df = df_cases_modified.groupBy("muslim_defendant").count()
jp_df.show()

+------------------+-------+
|  muslim_defendant|  count|
+------------------+-------+
|          1 muslim| 413284|
|-9999 missing name|   8196|
|       0 nonmuslim|2413917|
|     -9998 unclear| 200300|
+------------------+-------+



In [ ]:
jp_df = df_cases_modified.groupBy("muslim_petitioner").count()
jp_df.show()

In [ ]:
jp_df = df_cases_modified.groupBy("muslim_adv_def").count()
jp_df.show()

In [ ]:
jp_df = df_cases_modified.groupBy("muslim_adv_pet").count()
jp_df.show()

In [92]:
jp_df = df_cases_modified.groupBy("act_s").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+--------------------------+-------+
|act_s                     |count  |
+--------------------------+-------+
|Code of Criminal Procedure|586658 |
|The Indian Penal Code     |2449039|
+--------------------------+-------+

2


In [93]:
jp_df = df_cases_modified.groupBy("section_s").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------+------+
|section_s|count |
+---------+------+
|467      |23370 |
|296      |49    |
|451      |4725  |
|125      |206838|
|7        |2398  |
|124      |194   |
|447      |6645  |
|51       |347   |
|475      |57    |
|307      |46710 |
|205      |292   |
|334      |53    |
|169      |119   |
|272      |1534  |
|462      |13    |
|442      |20    |
|232      |515   |
|15       |599   |
|383      |50    |
|448      |5496  |
|54       |314   |
|234      |422   |
|155      |1183  |
|483      |14    |
|317      |336   |
|154      |169   |
|132      |114   |
|428      |125   |
|200      |903   |
|388      |338   |
|495      |500   |
|11       |310   |
|101      |42    |
|433      |26    |
|279      |92174 |
|415      |161   |
|422      |19    |
|361      |50    |
|138      |49355 |
|387      |2808  |
|323      |81251 |
|351      |27    |
|69       |61    |
|29       |640   |
|419      |1467  |
|309      |2506  |
|424      |101   |
|42       |196   |
|112      |31    |
|434      |1

In [ ]:
df_cases_modified.show()

+----------+----------------+-----------------+--------------+--------------+------------+-------------------+---------------------+---------+------------------------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+---------+--------------------+--------------------+----------------------------+
|state_code|female_defendant|female_petitioner|female_adv_def|female_adv_pet|billable_ipc|female_judge_filing|female_judge_decision|judgediff|pendency_decision_filing|pendency_first_filing|muslim_defendant|muslim_petitioner|muslim_adv_def|muslim_adv_pet|court_details|state_district|pendency_model_multiyear|filing_judge_tenure_at_filing|decision_judge_tenure_at_decision|               act_s|section_s|         type_name_s|    judge_position_s|number_sections_ipc_category|
+----------+----------------+-----------------+---

### Save as one file

In [ ]:
df_cases_modified.coalesce(1).write.csv("automl_2010to2018_final_criminal.csv", header=True, mode="overwrite")

In [ ]:
!cat /content/automl_2010to2018_final_criminal.csv/*.csv > /content/automl_2010to18_final_criminal.csv

In [ ]:
!cp /content/automl_2010to18_final_criminal.csv /content/drive/MyDrive/Law/Model2abbaselineData_2010to2018_merged_criminal_regr.csv